In [1]:
#### new code use image with spine  - 21-09-24

import numpy as np
import vtkplotlib as vpl
from stl import mesh
import pandas as pd
import csv
#test copy
ribcage_mesh = mesh.Mesh.from_file(r"BodyParts3D_Rib_cage.stl")


In [2]:
fracture_data = []
import pandas as pd
df = pd.read_excel('CAPSTONE_OUTCOMES2.xlsx')
df_cleaned = df.drop(columns=['side_injured', 'left_ribs', 'right_ribs', 'record_id', 'study_id','rib_fracture_mapping_complete'])
# Melt the DataFrame to bring all Rib values under one column
df_melted = pd.melt(df_cleaned, id_vars=['Primary_outcome'], var_name='Rib_Fracture', value_name='Fracture_Value')

In [3]:
# Extract the Rib number from the 'Rib_Fracture' column
df_melted['Rib_number'] = df_melted['Rib_Fracture'].str.extract(r'Rib(\d+)_\d+').astype(int)

In [4]:
# Drop the original 'Rib_Fracture' column as it's no longer needed
df_final = df_melted.drop(columns=['Rib_Fracture'])
df_final_cleaned = df_final.dropna(subset=['Fracture_Value'])

In [5]:
fracture_data = df_final_cleaned.rename(columns={
    'Primary_outcome': 'rib_colour',
    'Rib_number': 'rib_number',
    'Fracture_Value': 'angle'
})

In [6]:
fracture_data = fracture_data[['rib_number', 'angle', 'rib_colour']]

In [7]:
import numpy as np
def extract_rib_centroids(ribcage_mesh, rib_number):
    ribcage_points = ribcage_mesh.points.astype(float)
    z_min, z_max = np.min(ribcage_points[:, 2]), np.max(ribcage_points[:, 2])
    ribs_z_positions = np.linspace(z_min, z_max, num=12)
    
    rib_number = int(rib_number)
    if rib_number < 1 or rib_number > 12:
        raise ValueError("Rib number must be between 1 and 12.")
    
    rib_z = ribs_z_positions[rib_number - 1]
    rib_vertices = ribcage_points[(ribcage_points[:, 2] > rib_z - 2.5) & 
                                  (ribcage_points[:, 2] < rib_z + 2.5)]
    
    rib_centroid = np.mean(rib_vertices, axis=0)
    return rib_centroid, rib_vertices
def separate_rib_and_spine(ribcage_points, spine_threshold=5):
    """
    Separates the rib points from the spine points in the ribcage mesh.
    Assumes the spine is in the center, and ribs are more spread out along the x-axis.
    Parameters:
    ribcage_points (ndarray): The points of the ribcage mesh.
    spine_threshold (float): Threshold value to filter out spine vertices (based on x-axis).
    Returns:
    rib_points (ndarray): Points belonging to ribs.
    spine_points (ndarray): Points belonging to the spine.
    """
    # Assume that spine points have small x-values (close to the center)
    rib_points = ribcage_points[np.abs(ribcage_points[:, 0]) > spine_threshold]
    spine_points = ribcage_points[np.abs(ribcage_points[:, 0]) <= spine_threshold]
    return rib_points, spine_points
def calculate_rib_radius(rib_centroid, rib_vertices):
    """
    Calculates the mean radius of the rib based on its vertices and centroid.
    Parameters:
    rib_centroid (ndarray): The centroid of the rib.
    rib_vertices (ndarray): Vertices of the rib.
    Returns:
    float: The calculated mean radius of the rib.
    """
    distances = np.linalg.norm(rib_vertices[:, :2] - rib_centroid[:2], axis=1)
    mean_radius = np.mean(distances)
    
    return mean_radius

def angle_to_3d_coordinates(rib_number, angle, ribcage_mesh):
    """
    Maps a fracture angle to 3D coordinates on the surface of a specific rib.
    Parameters:
    rib_number (int): The rib number (1-12).
    angle (float): The angle for fracture projection.
    ribcage_mesh: The ribcage mesh object.
    Returns:
    np.array: 3D coordinates of the fracture point on the rib surface.
    """
    # Ensure angle is a float
    angle = float(angle)
    # Get rib centroid and vertices
    rib_centroid, rib_vertices = extract_rib_centroids(ribcage_mesh, rib_number)
    # Calculate the radius for this rib
    radius = calculate_rib_radius(rib_centroid, rib_vertices)
    # Project the fracture onto the rib surface, avoiding the spine
    fracture_point = project_fracture_to_rib_surface(rib_vertices, angle)
    return fracture_point


In [8]:
def project_fracture_to_rib_surface(rib_vertices, angle, spine_threshold=5):
    angle_rad = np.radians(angle)
    rib_vertices_filtered = rib_vertices[np.abs(rib_vertices[:, 0]) > spine_threshold]

    # Debugging: Print the filtered rib vertices
    #print(f"Filtered Rib Vertices Count: {rib_vertices_filtered.shape[0]}")
    if rib_vertices_filtered.shape[0] == 0:
        raise ValueError("No rib vertices available for projection.")

    distances = np.linalg.norm(rib_vertices_filtered[:, :2], axis=1)
    mean_radius = np.mean(distances)
    
    x = mean_radius * np.cos(angle_rad)
    y = mean_radius * np.sin(angle_rad)
    z = np.mean(rib_vertices_filtered[:, 2])
    
    projected_point = np.array([x, y, z])
    
    rib_vertices_3d = rib_vertices_filtered[:, :3]
    closest_vertex = rib_vertices_3d[np.argmin(np.linalg.norm(rib_vertices_3d - projected_point, axis=1))]

    # Debugging: Print the projected point and closest vertex
    #print(f"Projected Point: {projected_point}, Closest Vertex: {closest_vertex}")

    return closest_vertex


In [9]:
## write dataframe to csv, to remove index.
fracture_data.to_csv('newfileFracture.csv', index=False)

In [10]:
fracture_data = []
with open('newfileFracture.csv',newline='') as csvfile:
    # Use the csv.reader to read the file
    csvreader = csv.reader(csvfile)  
    # Skip the header if your CSV has one
    next(csvreader)
    # Convert each row into a tuple and append to the list
    for row in csvreader:
        fracture_data.append(tuple(row)) 

In [11]:
fracture_points = []
# Function to visualize ribcage mesh and fractures
def visualize_ribcage_with_fractures(ribcage_mesh, fracture_points):
    vpl.mesh_plot(ribcage_mesh)  # Plot the ribcage mesh
    
    # Plot the fracture points with the corresponding color
    for fracture_point, rib_colour in fracture_points:
        vpl.scatter(fracture_point, color=rib_colour, radius=2)  # Use the rib_colour from fracture_points
    
    vpl.show()


In [12]:

# Check and clean fracture_data
cleaned_fracture_data = []
for entry in fracture_data:
    # Skip headers or any entries that are not numeric
    if isinstance(entry[0], str) and entry[0].lower() in ['rib_number', 'angle', 'rib_colour']:
        print(f"Skipping header entry: {entry}")
        continue

    # Ensure each entry has at least two elements for rib_number and angle
    if len(entry) < 3:
        print(f"Skipping invalid entry (too few elements): {entry}")
        continue

    rib_number = entry[0]
    angle = entry[1]
    
    # Check if rib_number and angle can be converted to float
    try:
        rib_number_float = float(rib_number)  # Convert to float for processing
        angle_float = float(angle)  # Convert to float for processing
        cleaned_fracture_data.append((rib_number_float, angle_float, entry[2] if len(entry) > 2 else None))
    except ValueError:
        print(f"Skipping entry with invalid numbers (Rib: {rib_number}, Angle: {angle})")
        continue

# Now process the cleaned data
for entry in cleaned_fracture_data:
    try:
        # Unpack the cleaned values
        rib_number = entry[0]
        angle = entry[1]
        rib_colour = entry[2]  # This might be None if not provided

        # Convert Rib_colour values to corresponding colors
        if rib_colour == "Yes":
            color = "#FDE725FF"
        elif rib_colour == "No":
            color = "#541352FF"
        else:  # Handles both 'NaN' and other undefined cases
            color = "#2f9aa0FF"

        # Calculate the 3D coordinates for the fracture point
        fracture_point = angle_to_3d_coordinates(rib_number, angle, ribcage_mesh)
        
        # Append the fracture point along with its color to fracture_points
        fracture_points.append((fracture_point, color))

    except Exception as e:
        print(f"Error processing Rib {rib_number} with Angle {angle}: {e}")

# Visualize the ribcage with fracture points and colors
visualize_ribcage_with_fractures(ribcage_mesh, fracture_points)
